# Continuous Acoustic Emission features extraction

In [ ]:
import numpy as np

def clearance_factor(signal: np.ndarray) -> float:
    return np.max(np.abs(signal)) / (np.mean(np.sqrt(np.abs(signal))) ** 2)

def crest_factor(signal: np.ndarray) -> float:
    return np.max(np.abs(signal)) / np.sqrt(np.mean(signal**2))

def energy(signal: np.ndarray, samplerate: float) -> float:
    return 1 / samplerate * np.sum(signal**2)

def impulse_factor(signal: np.ndarray) -> float:
    return np.max(np.abs(signal)) / np.mean(np.abs(signal))

def kurtosis(signal: np.ndarray) -> float:
    def central_moment(order: int):
        return np.mean((signal - signal.mean()) ** order)

    return central_moment(4) / central_moment(2) ** 2


def partial_power(
    spectrum: np.ndarray, samplerate: float, fmin: float, fmax: float
) -> float:
    ps = np.abs(spectrum) ** 2
    n = len(ps)
    n_lower = math.floor(2 * n * fmin / samplerate)
    n_upper = math.floor(2 * n * fmax / samplerate)
    return np.sum(ps[n_lower:n_upper]) / np.sum(ps)


def peak_amplitude(signal: np.ndarray) -> float:
    return np.max(np.abs(signal))

def rms(signal: np.ndarray) -> float:
    return np.sqrt(np.mean(signal**2))

def shape_factor(signal: np.ndarray) -> float:
    return np.sqrt(np.mean(signal**2)) / np.mean(np.abs(signal))

def skewness(signal: np.ndarray) -> float:
    def central_moment(order: int):
        return np.mean((signal - signal.mean()) ** order)

    return central_moment(3) / central_moment(2) ** (3 / 2)


def spectral_centroid(spectrum: np.ndarray, samplerate: float):
    ps = np.abs(spectrum) ** 2
    ps_sum = 0.0
    ps_sum_weighted = 0.0
    for i, magnitude in enumerate(ps):
        ps_sum += magnitude
        ps_sum_weighted += magnitude * i
    return 0.5 * samplerate / (len(ps) - 1) * (ps_sum_weighted / ps_sum)

def spectral_entropy(spectrum: np.ndarray):
    ps = np.abs(spectrum) ** 2
    ps_sum = np.sum(ps)
    if ps_sum == 0.0:
        return 0.0
    p = ps / ps_sum
    p = p[p != 0]
    return -np.sum(p * np.log2(p)) / np.log2(len(ps))

def spectral_flatness(spectrum: np.ndarray):
    def geometric_mean(x):
        if np.any(x == 0):
            return 0.0
        return np.exp(np.mean(np.log(x)))

    ps = np.abs(spectrum) ** 2
    return geometric_mean(ps) / np.mean(ps)

def _spectral_centroid(spectrum: np.ndarray, samplerate: float):
    ps = np.abs(spectrum) ** 2
    ps_sum = 0.0
    ps_sum_weighted = 0.0
    for i, magnitude in enumerate(ps):
        ps_sum += magnitude
        ps_sum_weighted += magnitude * i
    return 0.5 * samplerate / (len(ps) - 1) * (ps_sum_weighted / ps_sum)


def spectral_kurtosis(spectrum: np.ndarray, samplerate: float):
    f_centroid = _spectral_centroid(spectrum, samplerate)
    ps = np.abs(spectrum) ** 2
    ps_sum = 0.0
    ps_sum_weighted_2 = 0.0
    ps_sum_weighted_4 = 0.0
    for i, magnitude in enumerate(ps):
        f = 0.5 * samplerate / (len(ps) - 1) * i
        ps_sum += magnitude
        ps_sum_weighted_2 += magnitude * (f - f_centroid) ** 2
        ps_sum_weighted_4 += magnitude * (f - f_centroid) ** 4
    return (ps_sum_weighted_4 / ps_sum) / np.sqrt(ps_sum_weighted_2 / ps_sum) ** 4

def spectral_peak_frequency(spectrum: np.ndarray, samplerate: float) -> float:
    peak_bin = np.argmax(spectrum**2)
    return 0.5 * samplerate / (len(spectrum) - 1) * peak_bin

def spectral_rolloff(spectrum: np.ndarray, samplerate: float, rolloff: float):
    ps = np.abs(spectrum) ** 2
    ps_sum_rolloff = rolloff * np.sum(ps)
    ps_sum = 0.0
    for i, magnitude in enumerate(ps):
        ps_sum += magnitude
        if ps_sum >= ps_sum_rolloff:
            return 0.5 * samplerate / (len(ps) - 1) * i
    return 0.5 * samplerate


def spectral_skewness(spectrum: np.ndarray, samplerate: float):
    f_centroid = _spectral_centroid(spectrum, samplerate)
    ps = np.abs(spectrum) ** 2
    ps_sum = 0.0
    ps_sum_weighted_2 = 0.0
    ps_sum_weighted_3 = 0.0
    for i, magnitude in enumerate(ps):
        f = 0.5 * samplerate / (len(ps) - 1) * i
        ps_sum += magnitude
        ps_sum_weighted_2 += magnitude * (f - f_centroid) ** 2
        ps_sum_weighted_3 += magnitude * (f - f_centroid) ** 3
    return (ps_sum_weighted_3 / ps_sum) / np.sqrt(ps_sum_weighted_2 / ps_sum) ** 3


def spectral_variance(spectrum: np.ndarray, samplerate: float):
    f_centroid = _spectral_centroid(spectrum, samplerate)
    ps = np.abs(spectrum) ** 2
    ps_sum = 0.0
    ps_sum_weighted = 0.0
    for i, magnitude in enumerate(ps):
        f = 0.5 * samplerate / (len(ps) - 1) * i
        ps_sum += magnitude
        ps_sum_weighted += magnitude * (f - f_centroid) ** 2
    return ps_sum_weighted / ps_sum


def zero_crossing_rate(signal: np.ndarray, samplerate: float) -> int:
    if len(signal) == 0:
        return 0
    crossings = 0
    was_positive = signal[0] >= 0
    for value in signal[1:]:
        is_positive = value >= 0
        if was_positive != is_positive:
            crossings += 1
        was_positive = is_positive
    return samplerate * crossings / len(signal)